In [20]:
%matplotlib inline
import cv2
import IPython
import time
from io import BytesIO
import numpy as np
import PIL

In [21]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()

    #flip image for natural viewing
    frame = cv2.flip(frame, 1)
    
    #frame = modify_frame(frame)

    return frame


#Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    #Create binary stream object
    f = BytesIO()

    #Convert array to binary stream object
    PIL.Image.fromarray(a).save(f, fmt)

    return IPython.display.Image(data=f.getvalue())



def modify_frame(frame):
    kernel = np.ones((3,3),np.uint8)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # define range of skin color in HSV
    lower_skin = np.array([0,20,70], dtype=np.uint8)
    upper_skin = np.array([20,255,255], dtype=np.uint8)

    #extract skin colur image 
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    #extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations = 4)
    
    #blur the image
    frame = cv2.GaussianBlur(mask,(7,7),50) 

    return frame



def get_region_of_interest(frame):
    start_point = (760, 110)
    end_point = (1200,530)
    color = (255, 0, 0)
    thickness = 1
    frame = cv2.rectangle(frame, start_point, end_point, color, thickness)
    return frame[start_point[1]:end_point[1], start_point[0]: end_point[0]]

In [22]:
cam = cv2.VideoCapture(0)

d = IPython.display.display("", display_id=1)
d2 = IPython.display.display("", display_id=2)

while True:
    try:
        t1 = time.time()
        frame = get_frame(cam)
        
        #roi = get_region_of_interest(frame)
        
        t_mid = time.time()
        record_image = 0
        if (t_mid - t1 > 0) and (record_image == 1):
            name = "images/"+ str(int(t_mid)) + ".jpg"
            cv2.imwrite(name,roi)

        im = array_to_image(frame)

        d.update(im)

        t2 = time.time()

        s = f"""{int(1/(t2-t1))} FPS"""
        d2.update( IPython.display.HTML(s) )
    except KeyboardInterrupt:
        print()
        cam.release()
        IPython.display.clear_output()
        print ("Stream stopped")
        break

Stream stopped


In [39]:
import os

os.listdir('images/skin_color/')

['akhil_1587654474.jpg',
 'akhil_1587654475.jpg',
 'akhil_1587654477.jpg',
 'akhil_1587654476.jpg',
 'akhil_1587654466.jpg',
 '.DS_Store',
 'akhil_1587654473.jpg',
 'akhil_1587654467.jpg',
 'akhil_1587654478.jpg']

In [57]:
roi_1_hsv_values = []
roi_2_hsv_values = []
for image_path in os.listdir('images/skin_color/'):
    if image_path[0] == ".":
        continue
    path = 'images/skin_color/'+image_path
    # Using cv2.imread() method 
    img = cv2.imread(path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = hsv[:,:,0], hsv[:,:,1], hsv[:,:,2]
    if image_path[:5] == 'roi_1':
        roi_1_hsv_values.append((h, s, v))
    else:
        roi_2_hsv_values.append((h, s, v))
    


In [73]:
roi_1_h = np.stack(tuple([itr[0] for itr in roi_1_hsv_values]), axis = 0)
roi_1_s = np.stack(tuple([itr[1] for itr in roi_1_hsv_values]), axis = 0)
roi_1_v = np.stack(tuple([itr[2] for itr in roi_1_hsv_values]), axis = 0)

roi_2_h = np.stack(tuple([itr[0] for itr in roi_2_hsv_values]), axis = 0)
roi_2_s = np.stack(tuple([itr[1] for itr in roi_2_hsv_values]), axis = 0)
roi_2_v = np.stack(tuple([itr[2] for itr in roi_2_hsv_values]), axis = 0)

In [85]:
offsetLowThreshold = 80
offsetHighThreshold = 30

h_low_threshold = int(max(0, min(roi_1_h.mean(), roi_2_h.mean()) - offsetLowThreshold))
h_high_threshold = int(min(255, max(roi_1_h.mean(), roi_2_h.mean()) + offsetHighThreshold))

s_low_threshold = int(max(0, min(roi_1_s.mean(), roi_2_s.mean()) - offsetLowThreshold))
s_high_threshold = int(min(255, max(roi_1_s.mean(), roi_2_s.mean()) + offsetHighThreshold))

In [87]:
h_high_threshold, h_low_threshold

(37, 0)

In [88]:
s_high_threshold, s_low_threshold

(148, 33)